In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow.keras.preprocessing.sequence as sequence

In [ ]:
plot_movie = pd.read_csv('wiki_movie_plots_deduped.csv')
all_word_in_plot = ""
for word in range(len(plot_movie['Plot'])//720):
    all_word_in_plot += ' ' + plot_movie['Plot'][word]

In [3]:
data = """DIO: Tsugi wa Jotaro, kisama da!
Jotaro: Yarou… DIO!
DIO: Ho… mukatta kuruno ka? Nigetsu ni kono DIO ni chikazuite kuruno ka? Sekkaku sofu no Josefu ga watashi no Za Warudo no shotai wo. Shiken shuryu chaimu chokuzen made mondai yo toitte iru jukensee ne you na? Kisshi koita kibun de wo shietekure ta to yuu no ni?
Jotaro: Chikadzu kanaka teme wo buchi no me tenain de na.
DIO: Hoho! Dewa juubun chikazukanai youi.
Jotaro: Ora!
DIO: Noroi, noroi! Za Warudo wa saikyou no Sutando da. Jikan wa tomezetomo, supiido to paowa to te omae no Suta Purachina yoryuu enna no towa!
Jotaro: Ore no Suta Purachina to onaji taipu wo Sutando nara. Enkyori enai kenai da, paowa to semitsu na bokina dekiru
Jotaro and Dio Japanese Conversation-DIO: Tsugi wa Jotaro, kisama da!
Jotaro: Yarou… DIO!
DIO: Ho… mukatta kuruno ka? Niget
"""
# data = all_word_in_plot
tokenizer = Tokenizer()
corpus = data.lower().split("\n")

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

print(tokenizer.word_index)
print(input_sequences[:10])


{'dio': 1, 'no': 2, 'jotaro': 3, 'to': 4, 'wa': 5, 'da': 6, 'wo': 7, 'kuruno': 8, 'ka': 9, 'ni': 10, 'na': 11, 'tsugi': 12, 'kisama': 13, 'yarou…': 14, 'ho…': 15, 'mukatta': 16, 'za': 17, 'warudo': 18, 'de': 19, 'noroi': 20, 'sutando': 21, 'paowa': 22, 'suta': 23, 'purachina': 24, 'nigetsu': 25, 'kono': 26, 'chikazuite': 27, 'sekkaku': 28, 'sofu': 29, 'josefu': 30, 'ga': 31, 'watashi': 32, 'shotai': 33, 'shiken': 34, 'shuryu': 35, 'chaimu': 36, 'chokuzen': 37, 'made': 38, 'mondai': 39, 'yo': 40, 'toitte': 41, 'iru': 42, 'jukensee': 43, 'ne': 44, 'you': 45, 'kisshi': 46, 'koita': 47, 'kibun': 48, 'shietekure': 49, 'ta': 50, 'yuu': 51, 'chikadzu': 52, 'kanaka': 53, 'teme': 54, 'buchi': 55, 'me': 56, 'tenain': 57, 'hoho': 58, 'dewa': 59, 'juubun': 60, 'chikazukanai': 61, 'youi': 62, 'ora': 63, 'saikyou': 64, 'jikan': 65, 'tomezetomo': 66, 'supiido': 67, 'te': 68, 'omae': 69, 'yoryuu': 70, 'enna': 71, 'towa': 72, 'ore': 73, 'onaji': 74, 'taipu': 75, 'nara': 76, 'enkyori': 77, 'enai': 78, '

In [4]:
print(total_words)

87


In [5]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(sequence.pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [6]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(total_words,8))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(max_sequence_len-1, return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(max_sequence_len-1)))
model.add(tf.keras.layers.Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(xs, ys, epochs=500, verbose=1)

Epoch 1/500
5/5 [==============================] - 6s 33ms/step - loss: 4.4664 - accuracy: 0.0075
Epoch 2/500
5/5 [==============================] - 0s 31ms/step - loss: 4.4587 - accuracy: 0.0677
Epoch 3/500
5/5 [==============================] - 0s 31ms/step - loss: 4.4511 - accuracy: 0.0677
Epoch 4/500
5/5 [==============================] - 0s 33ms/step - loss: 4.4353 - accuracy: 0.0677
Epoch 5/500
5/5 [==============================] - 0s 33ms/step - loss: 4.3986 - accuracy: 0.0677
Epoch 6/500
5/5 [==============================] - 0s 31ms/step - loss: 4.3271 - accuracy: 0.0677
Epoch 7/500
5/5 [==============================] - 0s 31ms/step - loss: 4.3134 - accuracy: 0.0677
Epoch 8/500
5/5 [==============================] - 0s 34ms/step - loss: 4.2886 - accuracy: 0.0677
Epoch 9/500
5/5 [==============================] - 0s 32ms/step - loss: 4.2752 - accuracy: 0.0677
Epoch 10/500
5/5 [==============================] - 0s 32ms/step - loss: 4.2703 - accuracy: 0.0677
Epoch 11/500
5/5 [=

In [7]:
seed_text = "mukatta kuruno ka"
token_list = tokenizer.texts_to_sequences([seed_text])[0]
print(token_list)
token_list = sequence.pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

predicted = np.argmax(model.predict(token_list), axis=-1)
print(predicted)
for word, index in tokenizer.word_index.items():
    if index == predicted:
        print(word)
        break

[16, 8, 9]
1/1 [==============================] - 1s 1s/step
[9]
ka


In [8]:
next_word = 10
for _ in range(next_word):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = sequence.pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre') 
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""

    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 23ms/step
mukatta kuruno ka ka saikyou no sutando da jikan wa tomezetomo supiido to


In [9]:
model.save('./model_DIO.h5')

d:\1YearOne\Y3_S1\240-318_AI\MODEL_PROJ\venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
img = Image.open('./your_nxt_lns.jpg')
I1 = ImageDraw.Draw(img)
myFont = ImageFont.truetype('COOPBL.TTF',size=65)
I1.text((150, 800), "TEST test", font=myFont,fill=(255, 255, 255))
img.save('./your_nxt_lns_result.jpg')